In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer

from scipy import stats
import numpy as np
from scipy.stats import boxcox

In [4]:
holidays = pd.read_csv("holidays.csv")
holidays.head()

,date,type
0,1,0
1,5,4
2,12,4
3,42,0
4,43,0


In [6]:
dates = pd.read_csv("dates.csv")
dates.head()

,date,year,month,dayofmonth,dayofweek,dayofyear,weekofyear,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,year_weekofyear
0,365,1,1,1,2,1,1,1,True,False,True,False,True,False,101
1,366,1,1,2,3,2,1,1,False,False,False,False,False,False,101
2,367,1,1,3,4,3,1,1,False,False,False,False,False,False,101
3,368,1,1,4,5,4,1,1,False,False,False,False,False,False,101
4,369,1,1,5,6,5,1,1,False,False,False,False,False,False,101


In [8]:
dates.tail()

,date,year,month,dayofmonth,dayofweek,dayofyear,weekofyear,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,year_weekofyear
1315,1680,4,8,11,4,223,32,3,False,False,False,False,False,False,432
1316,1681,4,8,12,5,224,32,3,False,False,False,False,False,False,432
1317,1682,4,8,13,6,225,32,3,False,False,False,False,False,False,432
1318,1683,4,8,14,0,226,33,3,False,False,False,False,False,False,433
1319,1684,4,8,15,1,227,33,3,False,False,False,False,False,False,433


In [9]:
sample_submission = pd.read_csv("SampleSubmission.csv")
sample_submission.head()

,ID,target
0,year_week_425_store_1_category_0,0
1,year_week_426_store_1_category_0,0
2,year_week_427_store_1_category_0,1
3,year_week_428_store_1_category_0,3
4,year_week_429_store_1_category_0,2


In [10]:
stores = pd.read_csv("stores.csv")
stores.head()

,store_id,city,type,cluster
0,store_1,0,0,0
1,store_2,0,0,0
2,store_3,0,0,1
3,store_4,0,0,2
4,store_5,1,0,3


In [11]:
test = pd.read_csv("test.csv")
test.head()

,date,store_id,category_id,onpromotion
0,1627,store_1,category_24,0
1,1627,store_1,category_21,0
2,1627,store_1,category_32,0
3,1627,store_1,category_18,16
4,1627,store_1,category_26,0


In [13]:
train = pd.read_csv("train.csv")
train.head()

,date,store_id,category_id,target,onpromotion,nbr_of_transactions
0,365,store_1,category_24,0.0,0,0.0
1,365,store_1,category_21,0.0,0,0.0
2,365,store_1,category_32,0.0,0,0.0
3,365,store_1,category_18,0.0,0,0.0
4,365,store_1,category_26,0.0,0,0.0


In [1]:
import pandas as pd
import numpy as np
# Create a sample dataframe
data = {
'Age': [25, 30, 35, 40, 45],
'Income': [50000, 60000, 75000, 90000, 80000],
'Education': ['High School', 'Bachelor', 'Master', 'Ph.D.', 'Bachelor'],
'City': ['New York', 'San Francisco', 'Chicago', 'Los Angeles', 'Miami'],
'Gender': ['Male', 'Female', 'Male', 'Female', 'Male'],
'Productivity': [5, 4, 3, 2, 4]
}
df = pd.DataFrame(data)


In [7]:
df

,Age,Income,Education,City,Gender,Productivity
0,25,50000,High School,New York,Male,5
1,30,60000,Bachelor,San Francisco,Female,4
2,35,75000,Master,Chicago,Male,3
3,40,90000,Ph.D.,Los Angeles,Female,2
4,45,80000,Bachelor,Miami,Male,4


In [8]:
df_encoded = pd.get_dummies(df, columns=['Education', 'City', 'Gender'])
df_encoded

,Age,Income,Productivity,Education_Bachelor,Education_High School,Education_Master,Education_Ph.D.,City_Chicago,City_Los Angeles,City_Miami,City_New York,City_San Francisco,Gender_Female,Gender_Male
0,25,50000,5,False,True,False,False,False,False,False,True,False,False,True
1,30,60000,4,True,False,False,False,False,False,False,False,True,True,False
2,35,75000,3,False,False,True,False,True,False,False,False,False,False,True
3,40,90000,2,False,False,False,True,False,True,False,False,False,True,False
4,45,80000,4,True,False,False,False,False,False,True,False,False,False,True


In [14]:
from sklearn.preprocessing import StandardScaler
# Initialize the StandardScaler
scaler = StandardScaler()
# Fit and transform the selected columns
df_encoded[['Age', 'Income']] = scaler.fit_transform(df_encoded[['Age', 'Income']])
df_encoded

,Age,Income,Productivity,Education_Bachelor,Education_High School,Education_Master,Education_Ph.D.,City_Chicago,City_Los Angeles,City_Miami,City_New York,City_San Francisco,Gender_Female,Gender_Male,Income per Age
0,-1.414214e+00,-1.470294,5,False,True,False,False,False,False,False,True,False,False,True,1.039655
1,-7.071068e-01,-0.770154,4,True,False,False,False,False,False,False,False,True,True,False,1.089162
2,4.440892e-17,0.280056,3,False,False,True,False,True,False,False,False,False,False,True,inf
3,7.071068e-01,1.330266,2,False,False,False,True,False,True,False,False,False,True,False,1.881280
4,1.414214e+00,0.630126,4,True,False,False,False,False,False,True,False,False,False,True,0.445566


In [15]:
# Create a new feature 'Income per Age'
df_encoded['Income per Age'] = df_encoded['Income'] / df_encoded['Age']

In [17]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# Separate the target variable and the features
X = df_encoded.drop('Productivity', axis=1)
y = df_encoded['Productivity']

# Initialize the linear regression model
model = LinearRegression()

# Perform RFE with cross-validation
rfe = RFE(model, n_features_to_select=5)  # Select the top 3 features
fit = rfe.fit(X, y)  # Corrected: use X instead of x

# Print the selected features
selected_features = X.columns[fit.support_]
print('Selected Features:', selected_features)


Selected Features: Index(['Age', 'Income', 'Education_High School', 'City_New York',
       'Income per Age'],
      dtype='object')


In [18]:
# Create bins and labels for the single column
bins = [-float('inf'), -0.5, 0.5, float('inf')]  # Adjust the bin edges as needed
labels = ['Young', 'Mid_Career', 'Senior']

# Bin the Age column
df_encoded['Binned_Column'] = pd.cut(df_encoded['Age'], bins=bins, labels=labels, right=False)

# Print the updated DataFrame
df_encoded

,Age,Income,Productivity,Education_Bachelor,Education_High School,Education_Master,Education_Ph.D.,City_Chicago,City_Los Angeles,City_Miami,City_New York,City_San Francisco,Gender_Female,Gender_Male,Income per Age,Binned_Column
0,-1.414214e+00,-1.470294,5,False,True,False,False,False,False,False,True,False,False,True,1.039655e+00,Young
1,-7.071068e-01,-0.770154,4,True,False,False,False,False,False,False,False,True,True,False,1.089162e+00,Young
2,4.440892e-17,0.280056,3,False,False,True,False,True,False,False,False,False,False,True,6.306301e+15,Mid_Career
3,7.071068e-01,1.330266,2,False,False,False,True,False,True,False,False,False,True,False,1.881280e+00,Senior
4,1.414214e+00,0.630126,4,True,False,False,False,False,False,True,False,False,False,True,4.455664e-01,Senior
